In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
from tqdm import tqdm_notebook
from PIL import Image, ImageFilter, ImageChops, ImageOps

In [ ]:
!rm -r ../train
!rm -r ../test

In [ ]:
!mkdir ../train
!mkdir ../test
!mkdir ../train/0
!mkdir ../train/1


!mkdir ../test/0
!mkdir ../test/1

In [ ]:
data = pd.read_csv('/kaggle/input/train_labels.csv')

In [ ]:
data.head()

In [ ]:
one = data.query("label == 1")['id'].values
one_train = one[:len(one) * 4 //5]
one_test = one[len(one) * 4 //5:]

zero = data.query("label == 0")['id'].values
zero_train = zero[:len(zero) * 4 //5]
zero_test = zero[len(zero) * 4 //5:]

In [ ]:
ORIGINAL_SIZE = 96
CROP_SIZE = 32

def process(path, mode, label):

    
    colorImage = Image.open("../input/train/%s.tif" % (path))
    
    
    start_crop = (ORIGINAL_SIZE - CROP_SIZE) // 2
    end_crop = start_crop + CROP_SIZE
    colorImage = colorImage.crop((start_crop, start_crop, end_crop, end_crop))

    
    for I in range(4):
        colorImage = colorImage.transpose(Image.ROTATE_90)
        colorImage.save("../%s/%d/%s_%d.png" % (mode, label, path, I), "PNG")
        
    colorImage = colorImage.transpose(Image.FLIP_LEFT_RIGHT)

    for I in range(4):
        colorImage = colorImage.transpose(Image.ROTATE_90)
        colorImage.save("../%s/%d/%s_%d.png" % (mode, label, path, I+4), "PNG")
    
    colorImage = colorImage.transpose(Image.FLIP_TOP_BOTTOM)
    
    for I in range(4):
        colorImage = colorImage.transpose(Image.ROTATE_90)
        colorImage.save("../%s/%d/%s_%d.png" % (mode, label, path, I+8), "PNG")  

    colorImage = colorImage.transpose(Image.FLIP_LEFT_RIGHT)
    
    for I in range(4):
        colorImage = colorImage.transpose(Image.ROTATE_90)
        colorImage.save("../%s/%d/%s_%d.png" % (mode, label, path, I+12), "PNG")  
    
    
    '''    
    colorImage = Image.open("../input/train/%s.tif" % (path))
    start_crop = (ORIGINAL_SIZE - CROP_SIZE) // 2
    end_crop = start_crop + CROP_SIZE
    colorImage = colorImage.crop((start_crop, start_crop, end_crop, end_crop))

    colorImage.save("../%s/%d/%s.png" % (mode, label, path), "PNG")
    '''
    

In [ ]:
def myFunc(image):
    image = np.array(image)
    return image / 255

In [ ]:
for I in tqdm_notebook(one_train):
    process(I, "train", 1)
for I in tqdm_notebook(one_test):
    process(I, "test", 1)
for I in tqdm_notebook(zero_train):
    process(I, "train", 0)
for I in tqdm_notebook(zero_test):
    process(I, "test", 0)

In [ ]:
from keras.models import Model

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, Dense, GlobalAveragePooling2D

model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(32,32,3), pooling='avg', classes=2)

In [ ]:
x = model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
combined_model = Model(inputs=model.input, outputs=predictions)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator()
train_it = datagen.flow_from_directory('../train/', class_mode='binary', batch_size=128, target_size=(32,32))

In [ ]:
test_it = datagen.flow_from_directory('../test/', class_mode='binary', batch_size=128, target_size=(32,32))

In [ ]:
combined_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
combined_model.fit_generator(train_it, steps_per_epoch=128, epochs=5, validation_data=test_it, validation_steps=128)